In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("DataFrame Intro").getOrCreate()

df = spark.read.option("delimiter",",").\
            option("header","true").\
            option("inferSchema","true").\
            csv("../datasets/iris-dataset.txt")



indexer = StringIndexer(inputCol="class",outputCol="label")
df = indexer.fit(df).transform(df)
df = df.drop("class")


vec = VectorAssembler(inputCols=["sepal-length","sepal-width",
                                "petal-length","petal-width"],
                     outputCol="features")
df = vec.transform(df)
df = df.select("features","label")

(trainDF,testDF) = df.randomSplit([0.7,0.3])

rfClassifier = RandomForestClassifier()

model = rfClassifier.fit(trainDF)

sonucDF = model.transform(testDF)

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

res = evaluator.evaluate(sonucDF)
print(res)


#sonucDF.show(150)

0.9318181818181818
